In [ ]:
using PyPlot

In [ ]:
using HDF5

In [ ]:
#runid = "FokkerPlanck_exp_cos_dt1E-6_nt10"
#runid = "FokkerPlanck_exp_r_dt1E-6_nt100"
#runid = "FokkerPlanck_sinx4_dt1E-3_nt10"
#runid = "FokkerPlanck_sinx4siny4_dt1E-3_nt10"
#runid = "FokkerPlanck_sinx4sinyetc_dt1E-3_nt10"

In [ ]:
#runid = "FokkerPlanck_exp_cos_dt1E-6_nt10000"
#runid = "FokkerPlanck_exp_r_dt1E-6_nt10000"
#runid = "FokkerPlanck_sinx4_dt1E-3_nt10000"
#runid = "FokkerPlanck_sinx4siny4_dt1E-3_nt10000"
#runid = "FokkerPlanck_sinx4sinyetc_dt1E-3_nt10000"

In [ ]:
h5 = h5open(runid * ".h5", "r")

In [ ]:
nplot = size(h5["ω"],3)
#nplot = 11

In [ ]:
ω₀ = view(h5["ω"][:,:,1],     :, :, 1)
ω₁ = view(h5["ω"][:,:,nplot], :, :, 1)
ϕ₀ = view(h5["ϕ"][:,:,1],     :, :, 1)
ϕ₁ = view(h5["ϕ"][:,:,nplot], :, :, 1)
;

In [ ]:
h = zeros(nplot)
for n in 1:nplot
    h[n] = sum_kbn(h5["ω"][:,:,n] .* h5["ϕ"][:,:,n])
end

In [ ]:
ℰ = zeros(nplot)
for n in 1:nplot
    ℰ[n] = sum_kbn(h5["ω"][:,:,n] .* h5["ω"][:,:,n])
end

In [ ]:
using VortexCollisions

In [ ]:
gr = Grid2d(size(ω₀,1), size(ω₀,2))

In [ ]:
ft = FourierTransform(gr)

In [ ]:
ω̂₀ = get_trans(ft);
ω̂₁ = get_trans(ft);

In [ ]:
prfft!(ft, Array(ω₀), ω̂₀);
prfft!(ft, Array(ω₁), ω̂₁);

In [ ]:
figure(figsize=(12,5))

subplot(121)
pcolormesh(abs.(ω̂₀))
colorbar()
title("Initial Condition ω̂")

subplot(122)
pcolormesh(abs.(ω̂₁))
colorbar()
title("Final Solution ω̂")

In [ ]:
figure(figsize=(12,5))

subplot(121)
contour(ω₀, colors="k")
pcolormesh(ω₀)
colorbar()
title("Initial Condition ω")

subplot(122)
contour(ω₁, colors="k")
pcolormesh(ω₁)
colorbar()
title("Final Solution ω")

savefig(runid * "_ω.png", dpi=150)

In [ ]:
figure(figsize=(12,5))

subplot(121)
contour(ϕ₀, colors="k")
pcolormesh(ϕ₀)
colorbar()
title("Initial Condition ϕ")

subplot(122)
contour(ϕ₁, colors="k")
pcolormesh(ϕ₁)
colorbar()
title("Final Solution ϕ")

savefig(runid * "_ϕ.png", dpi=150)

In [ ]:
figure(figsize=(12,5))

subplot(121)
scatter(ϕ₀[:], ω₀[:])
title("Initial Condition ω(ϕ)")
xlabel("ϕ")
ylabel("ω")

subplot(122)
scatter(ϕ₁[:], ω₁[:])
title("Final Solution ω(ϕ)")
xlabel("ϕ")
ylabel("ω")

savefig(runid * "_scatter.png", dpi=150)

In [ ]:
figure(figsize=(8,4))
plot((h-h[1])/h[1])
xlabel("t")
ylabel("(h-h₀)/h₀")
title("Energy")
savefig(runid * "_energy.png", dpi=150)

In [ ]:
figure(figsize=(8,4))
plot((ℰ-ℰ[1])/ℰ[1])
xlabel("t")
ylabel("(ℰ-ℰ₀)/ℰ₀")
title("Enstrophy")
savefig(runid * "_enstrophy.png", dpi=150)

In [ ]:
close(h5)